# 유방암 데이터 분류: 머신러닝 워크플로우와 KNN 하이퍼파라미터 튜닝

이 노트북은 scikit-learn의 유방암 진단 데이터셋을 사용하여, 악성(malignant)과 양성(benign) 종양을 분류하는 머신러닝 모델을 구축하는 전체 과정을 다룹니다. 

특히 **K-최근접 이웃(K-Nearest Neighbors, KNN)** 알고리즘을 중심으로, 모델의 성능에 큰 영향을 미치는 **하이퍼파라미터(hyperparameter)**, 즉 최적의 이웃 수(k)를 찾는 과정을 시각화를 통해 알아봅니다.

## 머신러닝 프로젝트 5단계 워크플로우

1.  **데이터 준비**: 분석에 필요한 데이터를 불러오고 구조를 파악합니다. (실제 프로젝트에서는 데이터 수집, 결측치/이상치 처리, 스케일링, 특성 공학 등 데이터 전처리에 대부분의 시간을 할애합니다.)
2.  **데이터셋 분할**: 모델의 일반화 성능을 평가하기 위해 데이터를 훈련(train) 세트와 테스트(test) 세트로 나눕니다.
3.  **알고리즘 선택 및 학습**: 해결하려는 문제에 맞는 머신러닝 알고리즘을 선택하고, 훈련 세트를 사용하여 모델을 학습(fit)시킵니다.
4.  **예측**: 학습된 모델을 사용하여 테스트 세트의 결과를 예측(predict)합니다.
5.  **성능 평가**: 예측 결과와 실제 정답을 비교하여 모델의 성능을 객관적인 지표(예: 정확도)로 평가합니다.

### 1. 데이터 준비 및 탐색

In [ ]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
cancer = load_breast_cancer()

# 데이터셋 구조 확인 (Bunch 객체)
print("데이터셋 키:", cancer.keys())

In [ ]:
# 데이터의 각 부분 확인
print("
타겟 이름:", cancer['target_names']) # ['malignant', 'benign'] -> [악성, 양성]
print("
특성 이름 (상위 5개):", cancer['feature_names'][:5])

# 특성(X)과 타겟(y) 데이터 분리
X = cancer['data']
y = cancer['target']

print("
특성 데이터 형태:", X.shape)
print("타겟 데이터 형태:", y.shape)

# 타겟 데이터 샘플 (0: 악성, 1: 양성)
print("타겟 데이터 샘플:", y[:20])

### 2. 데이터셋 분할

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터를 훈련 세트와 테스트 세트로 분할합니다.
# random_state: 재현 가능하도록 난수 시드를 고정합니다.
# test_size: 전체 데이터 중 테스트 세트의 비율을 지정합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234, test_size=0.4)

print(f"훈련 데이터 형태: {X_train.shape}")
print(f"테스트 데이터 형태: {X_test.shape}")

### 3, 4, 5단계: KNN 모델 학습, 예측 및 평가 (최적의 k 찾기)

K-최근접 이웃(KNN)은 예측하려는 데이터와 가장 가까운 k개의 훈련 데이터 포인트를 찾아, 그 이웃들의 다수결 투표로 클래스를 결정하는 단순하고 직관적인 알고리즘입니다.

여기서 **k (이웃의 수)**는 사용자가 직접 지정해야 하는 **하이퍼파라미터**이며, k값에 따라 모델의 성능이 크게 달라질 수 있습니다. 최적의 k를 찾기 위해, k를 1부터 10까지 변화시키면서 모델의 성능 변화를 관찰해 보겠습니다.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# k값의 변화에 따른 훈련/테스트 점수를 저장할 리스트
train_scores = []
test_scores = []

# 이웃의 수를 1부터 10까지 변화시키며 모델 성능 측정
n_neighbors_range = range(1, 11)

for k in n_neighbors_range:
    # 모델 생성 및 학습
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    
    # 훈련 세트와 테스트 세트의 정확도(score) 기록
    train_scores.append(model.score(X_train, y_train))
    test_scores.append(model.score(X_test, y_test))

# 결과 출력
for k, train_score, test_score in zip(n_neighbors_range, train_scores, test_scores):
    print(f"k={k:2}, 훈련 점수: {train_score:.4f}, 테스트 점수: {test_score:.4f}")

#### 최적의 k 찾기 (시각화)

k값의 변화에 따른 훈련 점수와 테스트 점수를 그래프로 그려보면 최적의 k를 쉽게 찾을 수 있습니다.

- **훈련 점수**는 k가 작을수록 높게 나타나는 경향이 있습니다 (과대적합, overfitting).
- **테스트 점수**가 가장 높으면서, 훈련 점수와의 차이가 크지 않은 지점이 가장 이상적인 k값입니다.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(n_neighbors_range, train_scores, label="Train Score")
plt.plot(n_neighbors_range, test_scores, label="Test Score")
plt.xticks(n_neighbors_range)
plt.xlabel("Number of Neighbors (k)")
plt.ylabel("Accuracy")
plt.title("KNN: Accuracy vs. Number of Neighbors")
plt.legend()
plt.grid(True)
plt.show()

# 그래프를 보면 k=6 또는 k=7 근처에서 테스트 점수가 가장 높고 안정적인 것을 확인할 수 있습니다.

### 추가: 로지스틱 회귀 모델과 성능 비교

In [ ]:
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(max_iter=5000) # 반복 횟수를 늘려 모델이 수렴하도록 보장
model_lr.fit(X_train, y_train)

print("--- 로지스틱 회귀 모델 ---")
print(f"훈련 세트 정확도: {model_lr.score(X_train, y_train):.4f}")
print(f"테스트 세트 정확도: {model_lr.score(X_test, y_test):.4f}")